# Dinâmica simbólica em Python

Este notebook demonstra como calcular as matrizes **M**, **C**, **H** e **G** para um robô de cadeia aberta usando os módulos Python criados a partir do projeto em MATLAB. A estrutura separa o código em vários arquivos `.py` e o notebook atua apenas como orquestrador.

### 1. Clonar o repositório

Você pode clonar seu repositório usando o comando `!git clone` no Colab.

In [1]:
# Substitua esta URL pela URL do seu repositório Git
repo_url = "https://github.com/danisilva1110-spec/PFPython.git"

# Nome da pasta de destino (derivado automaticamente da URL)
repo_dir = repo_url.rstrip("/").split("/")[-1]
if repo_dir.endswith(".git"):
    repo_dir = repo_dir[:-4]

import sys
from pathlib import Path

current_repo = Path.cwd()
local_package = current_repo / "robot_dynamics"

# Se o notebook já estiver dentro do repositório, use-o diretamente
if local_package.exists():
    repo_path = current_repo
else:
    repo_path = current_repo / repo_dir
    if not repo_path.exists():
        # Clone o repositório apenas se ele ainda não existir
        !git clone {repo_url} {repo_dir}

# Garanta que o repositório esteja no sys.path para que os módulos sejam encontrados
sys.path.insert(0, str(repo_path.resolve()))

Cloning into 'PFPython'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 293 (delta 47), reused 8 (delta 2), pack-reused 192 (from 2)
Receiving objects: 100% (293/293), 407.60 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [2]:
from sympy import simplify, symbols

from robot_dynamics import (
    forward_kinematics,
    inertia_matrix,
    coriolis_matrix,
    centripetal_vector,
    gravity_vector,
    kinetic_energy,
    planar_two_dof,
)

### 2. Listar o conteúdo do repositório clonado
Após clonar (ou ao usar este notebook diretamente dentro do repositório), a variável `repo_path` aponta para a pasta onde o código está. Você pode inspecionar os arquivos com o comando abaixo.


In [3]:
# Substitua 'repo_path' pelo caminho do repositório se você tiver alterado as variáveis acima
!ls {repo_path}

main.ipynb  README.md  robot_dynamics


## Definição do robô

Criamos um manipulador planar de 2 DOF com parâmetros de Denavit-Hartenberg padrão e propriedades inerciais simbólicas.

In [4]:
(links, q, qd, g_vec) = planar_two_dof()
q1, q2 = q
dq1, dq2 = qd

# Caso seja necessário ajustar os parâmetros simbólicos, basta usar subs nos objetos retornados.

## Cinemática direta

As transformadas homogêneas de cada elo são acumuladas com base nos parâmetros DH fornecidos.

In [5]:
transforms = forward_kinematics(links, q)
transforms

[Matrix([
 [cos(q1), -sin(q1), 0, l1*cos(q1)],
 [sin(q1),  cos(q1), 0, l1*sin(q1)],
 [      0,        0, 1,          0],
 [      0,        0, 0,          1]]),
 Matrix([
 [-sin(q1)*sin(q2) + cos(q1)*cos(q2), -sin(q1)*cos(q2) - sin(q2)*cos(q1), 0, l1*cos(q1) - l2*sin(q1)*sin(q2) + l2*cos(q1)*cos(q2)],
 [ sin(q1)*cos(q2) + sin(q2)*cos(q1), -sin(q1)*sin(q2) + cos(q1)*cos(q2), 0, l1*sin(q1) + l2*sin(q1)*cos(q2) + l2*sin(q2)*cos(q1)],
 [                                 0,                                  0, 1,                                                    0],
 [                                 0,                                  0, 0,                                                    1]])]

## Matrizes da dinâmica

A partir da energia cinética e potencial calculamos **M(q)**, **C(q,\dot q)**, o vetor centrípeto **H = C\,\dot q** e o vetor de gravidade **G(q)**.

In [6]:
M = simplify(inertia_matrix(links, q, qd))
C = simplify(coriolis_matrix(links, q, qd))
H = simplify(centripetal_vector(C, qd))
G = simplify(gravity_vector(links, q, g_vec))

M, C, H, G

(Matrix([
 [1.0*I1zz + 1.0*I2zz + 2.0*l1**2*m1 + 1.0*l1**2*m2 + 3.0*l1*l2*m2*cos(q2) + 2.0*l2**2*m2, 1.0*I2zz + 1.5*l1*l2*m2*cos(q2) + 2.0*l2**2*m2],
 [                                         1.0*I2zz + 1.5*l1*l2*m2*cos(q2) + 2.0*l2**2*m2,                        1.0*I2zz + 2.0*l2**2*m2]]),
 Matrix([
 [-1.5*dq2*l1*l2*m2*sin(q2), 1.5*l1*l2*m2*(-dq1 - dq2)*sin(q2)],
 [ 1.5*dq1*l1*l2*m2*sin(q2),                                 0]]),
 Matrix([
 [1.5*dq2*l1*l2*m2*(-2*dq1 - dq2)*sin(q2)],
 [            1.5*dq1**2*l1*l2*m2*sin(q2)]]),
 Matrix([
 [g*(3*l1*m1*cos(q1) + 2*l1*m2*cos(q1) + 3*l2*m2*cos(q1 + q2))/2],
 [                                      3*g*l2*m2*cos(q1 + q2)/2]]))

In [7]:
from sympy import Matrix, cos, sin

q1, q2 = q
dq1, dq2 = qd
link1, link2 = links
l1, l2 = link1.a, link2.a
m1, m2 = link1.mass, link2.mass
I1zz, I2zz = link1.inertia[2, 2], link2.inertia[2, 2]
g = -g_vec[1]

niku_subs = {I1zz: m1 * l1**2 / 3, I2zz: m2 * l2**2 / 3}

C1 = cos(q1)
C2 = cos(q2)
S2 = sin(q2)
C12 = cos(q1 + q2)

M_ref = Matrix(
    [
        [
            (1 / 3) * m1 * l1**2 + m2 * l1**2 + (1 / 3) * m2 * l2**2 + m2 * l1 * l2 * C2,
            (1 / 3) * m2 * l2**2 + (1 / 2) * m2 * l1 * l2 * C2,
        ],
        [
            (1 / 3) * m2 * l2**2 + (1 / 2) * m2 * l1 * l2 * C2,
            (1 / 3) * m2 * l2**2,
        ],
    ]
 )

C_ref = Matrix(
    [
        [-m2 * l1 * l2 * S2 * dq2, -m2 * l1 * l2 * S2 * (dq1 + dq2)],
        [m2 * l1 * l2 * S2 * dq1, 0],
    ]
 )

H_ref = C_ref * Matrix(qd)

G_ref = Matrix(
    [
        ((1 / 2) * m1 + m2) * g * l1 * C1 + (1 / 2) * m2 * g * l2 * C12,
        (1 / 2) * m2 * g * l2 * C12,
    ]
 )

M_niku_diff = simplify(M.subs(niku_subs) - M_ref)
C_niku_diff = simplify(C.subs(niku_subs) - C_ref)
H_niku_diff = simplify(H.subs(niku_subs) - H_ref)
G_niku_diff = simplify(G - G_ref)

M_niku_diff, C_niku_diff, H_niku_diff, G_niku_diff


(Matrix([
 [2.0*l1**2*m1 + 2.0*l1*l2*m2*cos(q2) + 2.0*l2**2*m2, l2*m2*(1.0*l1*cos(q2) + 2.0*l2)],
 [                   l2*m2*(1.0*l1*cos(q2) + 2.0*l2),                    2.0*l2**2*m2]]),
 Matrix([
 [-0.5*dq2*l1*l2*m2*sin(q2), -0.5*l1*l2*m2*(dq1 + dq2)*sin(q2)],
 [ 0.5*dq1*l1*l2*m2*sin(q2),                                 0]]),
 Matrix([
 [dq2*l1*l2*m2*(-1.0*dq1 - 0.5*dq2)*sin(q2)],
 [              0.5*dq1**2*l1*l2*m2*sin(q2)]]),
 Matrix([
 [1.0*g*(l1*m1*cos(q1) + l2*m2*cos(q1 + q2))],
 [                  1.0*g*l2*m2*cos(q1 + q2)]]))

## Energia cinética
Comparamos a energia cinética calculada a partir dos jacobianos com a expressão do livro do Niku, usando as mesmas substituições para as inércias e somando os termos via $	frac{1}{2} \dot q^T M_{	ext{ref}} \dot q$.

Os momentos de inércia usados aqui são dados em torno da origem DH de cada elo (ponta da junta) e são deslocados para o centro de massa na função `kinetic_energy` via eixo paralelo.

In [8]:
from sympy import Matrix, expand

T = simplify(kinetic_energy(links, q, qd))

T_niku = simplify(expand((Matrix(qd).T * M_ref * Matrix(qd))[0] / 2))
T_niku_diff = simplify(T.subs(niku_subs) - T_niku)

T, T_niku, T_niku_diff


(1.125*dq1**2*l1**2*m1 + dq1**2*(0.5*I1zz - 0.125*l1**2*m1) + 0.125*m2*(4*dq1**2*l1**2 + 12*dq1**2*l1*l2*cos(q2) + 9*dq1**2*l2**2 + 12*dq1*dq2*l1*l2*cos(q2) + 18*dq1*dq2*l2**2 + 9*dq2**2*l2**2) + (0.5*I2zz - 0.125*l2**2*m2)*(dq1 + dq2)**2,
 0.166666666666667*dq1**2*l1**2*m1 + dq1**2*l1**2*m2/2 + dq1**2*l1*l2*m2*cos(q2)/2 + 0.166666666666667*dq1**2*l2**2*m2 + 0.5*dq1*dq2*l1*l2*m2*cos(q2) + 0.333333333333333*dq1*dq2*l2**2*m2 + 0.166666666666667*dq2**2*l2**2*m2,
 1.0*dq1**2*l1**2*m1 + 1.0*dq1**2*l1*l2*m2*cos(q2) + 1.0*dq1**2*l2**2*m2 + 1.0*dq1*dq2*l1*l2*m2*cos(q2) + 2.0*dq1*dq2*l2**2*m2 + 1.0*dq2**2*l2**2*m2)

Os objetos retornados são simbólicos e podem ser avaliados numericamente ou exportados para geração automática de código (por exemplo, com `sympy.lambdify`).